In [2]:
# Initialize Otter
import otter
grader = otter.Notebook("hwk3-task1-spills.ipynb")

ModuleNotFoundError: No module named 'otter'

# Task 1: Visualizing oil spills in NY State

## Instructions

- First, update the following cell to have a link to *your* Homework 3 GitHub repository:

**UPDATE THIS LINK**
https://github.com/matteo-torres/eds220-hwk3


- Review the [complete rubric for this task](https://docs.google.com/document/d/1Ce4EO3MEHW5uLks4x8BECWlkim2bcmxB4f8Qo4z26z8/edit?usp=sharing) before starting.

- **Meaningful commits should be made every time you finish a major step.** We'll check your repository and view the commit history.

- Comment mindfully in a way that enriches your code. Comments should follow best practices.

- **Do not update the top cell with the `otter` import**, this is used internally for grading.

- Delete all the comments initially included in this notebook (ex: `# Your code here`).


## About the data
In this task you will use two datsets:

### First dataset: oil spills data

The first dataset contains information about [spilling incidents in the state of New York](https://data.ny.gov/Energy-Environment/Spill-Incidents/u44d-k5fk). 
You can find extra information and metadata for this dataset in the `NYSDEC_SpillIncidents_Overview.pdf` and `NYSDEC_SpillIncidents_DataDictionary.pdf` documents available at the portal.

### Second dataset: TIGER shapefile

The second dataset is the [the 2023 Counties (and equivalent) TIGER shapefile](https://www.census.gov/geographies/mapping-files/time-series/geo/tiger-line-file.2022.html#list-tab-790442341) from the Census Bureau. You can check the [metadata for all the TIGER shapefiles here](https://www.census.gov/programs-surveys/geography/technical-documentation/complete-technical-documentation/tiger-geo-line.html). 


## 1. File management
a. Create a `data/` directory inside your `eds220-hwk3` directory. If you download any data, it should be transferred into this folder. 

b. Add the `data/` directory to your `.gitignore` file and push your changes. This is IMPORTANT: the datasets we'll be working with are large and they will most likely jam your `git` workflow if you try to push them. Your `data/` directory should not be in your GitHub repository.

## 2. Datasets description
Read the metadata for both datsets and write a description about each one. Indicate the date sources and date of access.


*Your answer here.*

## 3. Import libraries

In [3]:
# Load packages
import os
import pandas as pd
import geopandas as gpd

## 4. Import data 
In the next cell:

- Import the NY spills data as a variable named `spills`. 
- Import the US counties data as a variable named `counties`.

If you are using file paths to access the data, these should be constructed using the `os` package.

In [4]:
# Create a repoducible file path for NY spills data
fp_spills = os.path.join("data", "Spill_Incidents_20241105.csv")

# Import NY spills data
spills = pd.read_csv(fp_spills)

# Create a repoducible file path for US counties data
fp_counties = os.path.join("data", "tl_2023_us_county", "tl_2023_us_county.shp")

# Import US counties data
counties = gpd.read_file(fp_counties)

## 5. Prepare spills data

This section refers to the `spills` dataframe. 
Check the outputs carefully, they will give you information you will need to use in the next exercises.

### 5.A. Exploration

In the next cells:

a. Check the dataframe's head.

b. Check the data types of the columns.

c. Check the unique values in the `material_family` column.

d. Check the unique values in the `county` column.

You can add other cells for preliminary data exploration.

In [ ]:
# a. Look at the first five rows of the dataframe
spills.head()

In [ ]:
# b. Look at the columns data types
spills.dtypes

In [ ]:
# c. Look at the unique values for "Material Family"
spills["Material Family"].unique()

In [ ]:
# d. Look at the unique values for "County"
spills["County"].unique()

In [ ]:
# Look at the number of rows/columns
spills.shape

<!-- BEGIN QUESTION -->

f. Use the next markdown cell to explain what information you obtained from the preliminary exploration. Use complete sentences.

<!-- END QUESTION -->

*Your answer here.*

### 5.B. Cleaning
a. Simplify column names as needed.

b. Update column data types as needed. If you're not sure if any data types are needed at this point, you may come back later to this cell.

In [5]:
# a. Convert columns to lower snake case
spills.columns = (spills.columns
                  .str.replace(" ", "_")
                  .str.lower())

# Verify
spills.columns

Index(['spill_number', 'program_facility_name', 'street_1', 'street_2',
       'locality', 'county', 'zip_code', 'swis_code', 'dec_region',
       'spill_date', 'received_date', 'contributing_factor', 'waterbody',
       'source', 'close_date', 'material_name', 'material_family', 'quantity',
       'units', 'recovered'],
      dtype='object')

In [6]:
# b. Convert "spill_date" to a datetime
spills["spill_date"] = pd.to_datetime(spills["spill_date"])

# Verify
spills["spill_date"].dtype

dtype('<M8[ns]')

### 5.C. Data selection

a. Select data about petroleum spills that took place between January 1st 2023, and October 31st 2023 (including both dates). Store it in a variable named `petr_23`. The data selection should occur in a single line. You may add cells as needed.

In [7]:
# a. Select for petroleum spills between 01/01/2023 - 10/31/23
petr_23 = spills[(spills["spill_date"] > "2023-01-01") & 
                 (spills["spill_date"] < "2023-10-23") & 
                 (spills["material_family"] == "Petroleum")]

<!-- BEGIN QUESTION -->

b. Use `print()` and f-strings to print messages verifying that `petr_23` only has data for 2023 and from January through October.
HINT: [pandas.Series.dt.year](https://pandas.pydata.org/docs/reference/api/pandas.Series.dt.year.html) and [pandas.Series.dt.month](https://pandas.pydata.org/docs/reference/api/pandas.Series.dt.month.html).

In [8]:
# b. Find the date range of petr_23
min_mth = petr_23["spill_date"].dt.month.min()
max_mth = petr_23["spill_date"].dt.month.max()
min_yr = petr_23["spill_date"].dt.year.min()
max_yr = petr_23["spill_date"].dt.year.max()

# Print the date range of petr_23 df
print(f"The petr_23 data consists of petroleum spill observations from {min_mth}/{min_yr} to {max_mth}/{max_yr}")

The petr_23 data consists of petroleum spill observations from 1/2023 to 10/2023


<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

c. Write tests using `assert` to verify that `petr_23` only has data for petroleum spills.

In [9]:
# c. Test that all rows of petr_23 have "Petroleum" in the "material_family" column
assert all(petr_23["material_family"] == "Petroleum"), "Non-petroleum spill detected!"
print("Only petroleum spills!")

Only petroleum spills!


<!-- END QUESTION -->

### 5.D. Data aggregation

Create new data frame named `spills_per_county` with the number of petroleum spills per county from January to October 2023 (i.e. use the `petr_23` dataframe).

The resulting `spills_per_county` dataframe must be as follows:

- Index: integer numbers starting from 0
- Data frame has two columns only:
    - Column one: county names, column name = `county`
    - Column two: number of petroleum spills per county in 2023, column name = `n_spills`

You may add cells if needed.


In [96]:
# Create new dataframe of spills per county with an index starting at 0
spills_per_county = pd.DataFrame(petr_23.groupby("county").count()["spill_number"]).reset_index().rename(columns = {"spill_number" : "n_spills"})

# Check new dataframe
spills_per_county

,county,n_spills
0,Bronx,2
1,Clinton,3
2,Cortland,1
3,Erie,3
4,Essex,1
5,Lewis,2
6,Monroe,2
7,Oneida,1
8,Onondaga,9
9,Oswego,1


## 6. Prepare counties data

This section refers to the `counties` geo-dataframe. 
Check the outputs carefully, they will give you context about the next exercises.

### 6.A. Exploration

In the next cells:

a. Check the geo-dataframe's head.

b. Check the data types of the columns.

c. Check the geo-dataframe's CRS.

d. Plot the geo-dataframe.

You can add any other cells of preliminary data exploration.

In [ ]:
# a. Your code here

In [ ]:
# b. Your code here

In [ ]:
# c. Your code here

In [ ]:
# d. Your code here

<!-- BEGIN QUESTION -->

f. Use the next markdown cell to explain what information you obtained from the preliminary exploration. Use complete sentences.

*Your answer here.*

<!-- END QUESTION -->

### 6.B. Cleaning

Simplify column names as needed.

In [ ]:
# Your code here


### 6.C. Data selection

Select all the counties for the state of New York in a new variable `ny_counties` and plot them.

In [ ]:
# Your code here


## 7 Merging

The goal of this section is to create a single dataframe with the number of spills per county and the corresponding geometries for each county.

### 7.A. Explore data

In the next cells:

a. Print the names of counties in `ny_counties`, without repetition.

b. Print the names of counties in `spills_per_county`, without repetition. Examine the differences in the outputs.


In [ ]:
# a.

In [ ]:
# b.




c. Which are the county names that are in `spills_per_county`, but not in `ny_counties`? Your answer should be a list of strings
HINT: You can use the `numpy` function [`setdiff1d()`](https://numpy.org/doc/stable/reference/generated/numpy.setdiff1d.html), pay attention to the example in the documentation. Store your answer in the `diff_names` variable. 



In [ ]:
diff_names_spills = ...
diff_names_spills


d. Which are the county names that are in `ny_counties`, but not in `spills_per_county`? Your answer should be a list of strings.


In [ ]:
diff_names_ny = ...
diff_names_ny




e. What is the number of spills in 'Tuscarora Indian Reservation'?


In [ ]:
# e. 


### 7.B. Data updates

In the next cell, update 'St Lawrence' to 'St. Lawrence' in `spills_per_county` data frame and use an `assert` statement to verify that the update was successful.

You may add cells if needed.

In [ ]:
# Your code here


### 7.C. Merge

In the following cells:

a. Use this cell to make any other updates needed for making an inner join of the `spills_per_county` and `ny_counties`. 

b. Over-write `ny_counties` as the  inner join of the `spills_per_county` and `ny_counties` dataframes.


In [ ]:
# a. Your code here


In [ ]:
# b. Your code here


From our previous exploration we know that Tuscarora Indian Reservation will not be in the join. Tuscarora Indian Reservation is located within Niagara county. 

c. Since the Tuscarora Indian Reservation does not appear in the `ny_counties` dataframe, add one spill to Niagara county. Add the cells you need to make this update **and** verify that the dataframe was updated.

In [ ]:
# c. Your code here

<!-- BEGIN QUESTION -->

## 8. Map
a. Create a choropleth map of the number of petroleum spills per county in the state of New York in 2023. 
Your map should be polished and have (at least) the following updates:
- a legend indicating the number of spills
- an updated color map
- a title 
- no axis
- an [annotation](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.annotate.html) with the data source (New York State Department of Environmental Conservation) and date of access.



In [ ]:
# a. Your code here

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

b. Write a figure description for the map, including tha names of the top 3 counties with the most petroleum spills. If you need to do additional data analysis to include relevant information from the data, you may include additional code cells here.

*Your answer here*